In [2]:
!pip install ultralytics opencv-python scikit-learn segmentation-models-pytorch --quiet
!pip install torch torchvision --quiet

import os
import shutil
import json
import time
import cv2
import torch
import torchvision
import numpy as np
from tqdm import tqdm
from ultralytics import YOLO
from sklearn.model_selection import train_test_split
from torchvision import transforms
import segmentation_models_pytorch as smp


In [3]:
dataset_dir = r"C:\Users\navlog\Dataset_7"
images_dir = os.path.join(dataset_dir, "images")
labels_dir = os.path.join(dataset_dir, "labels")
os.makedirs(images_dir, exist_ok=True)
os.makedirs(labels_dir, exist_ok=True)

coco_path = os.path.join(dataset_dir, "_annotations.coco.json")
with open(coco_path, 'r') as f:
    coco = json.load(f)

categories = {cat['id']: cat['name'] for cat in coco['categories']}
category_map = {cat_id: i for i, cat_id in enumerate(categories.keys())}
image_id_to_filename = {img['id']: img['file_name'] for img in coco['images']}
annotations_by_image = {}
for ann in coco['annotations']:
    image_id = ann['image_id']
    if image_id not in annotations_by_image:
        annotations_by_image[image_id] = []
    annotations_by_image[image_id].append(ann)

for image_id, anns in tqdm(annotations_by_image.items()):
    file_name = image_id_to_filename[image_id]
    txt_filename = os.path.splitext(file_name)[0] + ".txt"
    txt_path = os.path.join(labels_dir, txt_filename)
    with open(txt_path, 'w') as f:
        for ann in anns:
            cat_id = ann['category_id']
            segmentation = ann.get("segmentation", [])
            if not segmentation or not isinstance(segmentation[0], list):
                continue
            poly = segmentation[0]
            norm_poly = [str(round(p / 1280 if i % 2 == 0 else p / 720, 6)) for i, p in enumerate(poly)]
            f.write(f"{category_map[cat_id]} " + " ".join(norm_poly) + "\n")

for img in coco['images']:
    src = os.path.join(dataset_dir, img['file_name'])
    dst = os.path.join(images_dir, img['file_name'])
    if os.path.exists(src) and not os.path.exists(dst):
        shutil.move(src, dst)

image_files = [f for f in os.listdir(images_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
train_files, val_files = train_test_split(image_files, test_size=0.2, random_state=42)
for split, file_list in [('train', train_files), ('val', val_files)]:
    for subdir in ['images', 'labels']:
        os.makedirs(os.path.join(dataset_dir, split, subdir), exist_ok=True)
    for img_file in file_list:
        base = os.path.splitext(img_file)[0]
        img_src = os.path.join(images_dir, img_file)
        label_src = os.path.join(labels_dir, base + '.txt')
        shutil.copy(img_src, os.path.join(dataset_dir, split, 'images', img_file))
        if os.path.exists(label_src):
            shutil.copy(label_src, os.path.join(dataset_dir, split, 'labels', base + '.txt'))


100%|██████████| 3467/3467 [00:04<00:00, 745.90it/s]


In [4]:
data_yaml_path = os.path.join(dataset_dir, "data.yaml")
class_names = list(categories.values())
with open(data_yaml_path, 'w') as f:
    f.write(f"train: {os.path.join(dataset_dir, 'train/images').replace('\\', '/')}\n")
    f.write(f"val: {os.path.join(dataset_dir, 'val/images').replace('\\', '/')}\n")
    f.write(f"nc: {len(class_names)}\n")
    f.write(f"names: {class_names}\n")


In [5]:
yolo_model = YOLO('yolov8n-seg.pt')
yolo_model.train(data=data_yaml_path, epochs=100, imgsz=640, batch=16, device=0)


New https://pypi.org/project/ultralytics/8.3.174 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.155  Python-3.12.7 torch-2.5.1+cu121 CUDA:0 (Quadro RTX 4000, 8192MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\navlog\Dataset_7\data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, 

train: Scanning C:\Users\navlog\Dataset_7\train\labels... 2773 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2773/2773 [00:05<00:00, 496.44it/s]


train: New cache created: C:\Users\navlog\Dataset_7\train\labels.cache
val: Fast image access  (ping: 0.10.0 ms, read: 0.90.2 MB/s, size: 13.0 KB)


val: Scanning C:\Users\navlog\Dataset_7\val\labels... 694 images, 0 backgrounds, 0 corrupt: 100%|██████████| 694/694 [00:01<00:00, 409.25it/s]

val: New cache created: C:\Users\navlog\Dataset_7\val\labels.cache


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):   9%|▉         | 2/22 [00:01<00:14,  1.39it/s]

WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validati

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 22/22 [00:07<00:00,  2.88it/s]


                   all        694       1665      0.605      0.598      0.605      0.441      0.605      0.591      0.595      0.422
                  Burs        480        913      0.521      0.206      0.251     0.0962      0.549      0.209      0.244      0.071
                 Flash        124        132      0.761      0.788      0.787      0.502       0.73       0.75      0.741      0.331
              Key Hole          2          2      0.753          1      0.995      0.647      0.755          1      0.995      0.747
                  Weld        614        614      0.991      0.997      0.995      0.961      0.991      0.997      0.995      0.961
                  void          4          4          0          0          0          0          0          0          0          0
Speed: 0.1ms preprocess, 1.6ms inference, 0.0ms loss, 3.2ms postprocess per image
Results saved to runs\segment\train2


ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([2, 3, 4, 5, 7])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002AFA5978A10>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.0400

In [6]:
import torchvision
import segmentation_models_pytorch as smp
from torchvision.models.detection import MaskRCNN_ResNet50_FPN_Weights
from torchvision.models.segmentation import DeepLabV3_ResNet50_Weights
from torchvision import transforms

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Load trained YOLOv8 model
from ultralytics import YOLO
yolo = YOLO("runs/segment/train/weights/best.pt")

# Load Mask R-CNN with recommended pretrained weights
mask_rcnn = torchvision.models.detection.maskrcnn_resnet50_fpn(
    weights=MaskRCNN_ResNet50_FPN_Weights.DEFAULT
).to(device).eval()

# Load DeepLabV3+ with VOC-pretrained weights
deeplab = torchvision.models.segmentation.deeplabv3_resnet50(
    weights=DeepLabV3_ResNet50_Weights.DEFAULT
).to(device).eval()

# Load U-Net (binary) with ResNet34 encoder pretrained on ImageNet
unet = smp.Unet(
    encoder_name="resnet34", 
    encoder_weights="imagenet", 
    in_channels=3, 
    classes=1
).to(device).eval()

# Define transform
transform = transforms.Compose([transforms.ToTensor()])


In [7]:
import time

def measure_model(model_name, inference_fn, frame):
    start = time.time()
    output = inference_fn(frame)
    latency = time.time() - start
    return latency, output

def infer_maskrcnn(frame):
    with torch.no_grad():
        img_tensor = transform(frame).to(device)
        outputs = mask_rcnn([img_tensor])[0]
        return outputs

def infer_deeplab(frame):
    with torch.no_grad():
        img_tensor = transform(frame).unsqueeze(0).to(device)
        out = deeplab(img_tensor)['out']
        return torch.sigmoid(out)

def infer_unet(frame):
    with torch.no_grad():
        img_tensor = transform(frame).unsqueeze(0).to(device)
        out = unet(img_tensor)
        return torch.sigmoid(out)

# Replace webcam with video file
video_path = r"C:\Users\navlog\Dataset_7\Video_13.mp4"
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    raise IOError(f"Cannot open video file: {video_path}")

fps_results = {}
model_runs = {
    'YOLOv8n-seg': lambda f: yolo(f),
    'Mask R-CNN': infer_maskrcnn,
    'DeepLabV3+': infer_deeplab,
    'U-Net': infer_unet
}

for name, fn in model_runs.items():
    print(f"Running real-time test for {name}")
    frame_count, total_latency = 0, 0

    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)  # Restart video for each model

    while frame_count < 50:  # Limit to first 50 frames
        ret, frame = cap.read()
        if not ret:
            print("⚠️ Reached end of video or failed to read frame.")
            break
        frame = cv2.resize(frame, (640, 480))
        latency, _ = measure_model(name, fn, frame)
        total_latency += latency
        frame_count += 1

    if frame_count > 0:
        avg_latency = total_latency / frame_count
        fps_results[name] = {
            "latency": round(avg_latency, 4),
            "FPS": round(1 / avg_latency, 2)
        }
    else:
        fps_results[name] = {
            "latency": None,
            "FPS": None
        }
        print(f"⚠️ {name}: No frames processed.")

cap.release()

print("📊 FPS and Latency results:")
for model, metrics in fps_results.items():
    print(f"{model}: Latency = {metrics['latency']}s | FPS = {metrics['FPS']}")


Running real-time test for YOLOv8n-seg

0: 480x640 (no detections), 57.0ms
Speed: 5.3ms preprocess, 57.0ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 45.3ms
Speed: 22.6ms preprocess, 45.3ms inference, 2.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 62.6ms
Speed: 27.6ms preprocess, 62.6ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 21.9ms
Speed: 3.3ms preprocess, 21.9ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 20.8ms
Speed: 3.5ms preprocess, 20.8ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 20.2ms
Speed: 3.2ms preprocess, 20.2ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 20.2ms
Speed: 3.0ms preprocess, 20.2ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections),

In [8]:
import os
import time
import cv2
import torch
import numpy as np
from glob import glob

# ----------------------------
# Polygon to mask conversion
# ----------------------------

def load_yolo_segmentation_mask(label_path, img_shape):
    h, w = img_shape[:2]
    mask = np.zeros((h, w), dtype=np.uint8)

    if not os.path.exists(label_path):
        return mask

    with open(label_path, "r") as f:
        lines = f.readlines()

    for line in lines:
        parts = line.strip().split()
        if len(parts) < 7:  # class + at least 3 points (x1 y1 x2 y2 x3 y3)
            continue
        coords = np.array(list(map(float, parts[1:])))
        coords = coords.reshape(-1, 2)
        coords[:, 0] *= w  # x
        coords[:, 1] *= h  # y
        polygon = coords.astype(np.int32)
        cv2.fillPoly(mask, [polygon], color=1)

    return mask

# ----------------------------
# Inference Wrappers
# ----------------------------

def measure_model(model_name, inference_fn, frame):
    start = time.time()
    output = inference_fn(frame)
    latency = time.time() - start
    return latency, output

def infer_maskrcnn(frame):
    with torch.no_grad():
        img_tensor = transform(frame).to(device)
        outputs = mask_rcnn([img_tensor])[0]
        if 'masks' in outputs and len(outputs['masks']) > 0:
            mask = outputs['masks'][0, 0] > 0.5
        else:
            mask = torch.zeros((480, 640), dtype=torch.uint8)
        return mask.cpu().numpy()

def infer_deeplab(frame):
    with torch.no_grad():
        img_tensor = transform(frame).unsqueeze(0).to(device)
        out = deeplab(img_tensor)['out']
        mask = torch.sigmoid(out)[0, 0] > 0.5
        return mask.cpu().numpy()

def infer_unet(frame):
    with torch.no_grad():
        img_tensor = transform(frame).unsqueeze(0).to(device)
        out = unet(img_tensor)
        mask = torch.sigmoid(out)[0, 0] > 0.5
        return mask.cpu().numpy()

def infer_yolov8(frame):
    results = yolo(frame)
    if results[0].masks is not None:
        masks = results[0].masks.data
        combined_mask = torch.any(masks, dim=0) if masks.ndim == 3 else masks[0]
    else:
        combined_mask = torch.zeros((480, 640), dtype=torch.uint8)
    return combined_mask.cpu().numpy()

# ----------------------------
# IOU Computation
# ----------------------------

def compute_iou(pred_mask, true_mask):
    pred = pred_mask.astype(bool)
    true = true_mask.astype(bool)
    intersection = np.logical_and(pred, true).sum()
    union = np.logical_or(pred, true).sum()
    return intersection / union if union != 0 else 1.0

# ----------------------------
# Dataset Paths
# ----------------------------

val_image_dir = r"C:\Users\navlog\Dataset_7\val\images"
val_label_dir = r"C:\Users\navlog\Dataset_7\val\labels"

image_paths = sorted(glob(os.path.join(val_image_dir, "*.jpg")))  # or .png

# ----------------------------
# Model Definitions
# ----------------------------

fps_results = {}
model_runs = {
    'YOLOv8n-seg': infer_yolov8,
    'Mask R-CNN': infer_maskrcnn,
    'DeepLabV3+': infer_deeplab,
    'U-Net': infer_unet
}

# ----------------------------
# Evaluation
# ----------------------------

for name, fn in model_runs.items():
    print(f"\n🚀 Running evaluation for {name}")
    total_latency, total_iou, valid_frames = 0, 0, 0

    for i, img_path in enumerate(image_paths[:50]):
        frame = cv2.imread(img_path)
        frame = cv2.resize(frame, (640, 480))

        base_name = os.path.splitext(os.path.basename(img_path))[0]
        label_path = os.path.join(val_label_dir, base_name + ".txt")

        gt_mask = load_yolo_segmentation_mask(label_path, frame.shape)

        latency, pred_mask = measure_model(name, fn, frame)
        iou = compute_iou(pred_mask, gt_mask)

        total_latency += latency
        total_iou += iou
        valid_frames += 1

    if valid_frames > 0 and total_latency > 0:
        avg_latency = total_latency / valid_frames
        avg_iou = total_iou / valid_frames
        fps_results[name] = {
            "latency": round(avg_latency, 4),
            "FPS": round(1 / avg_latency, 2),
            "IoU": round(avg_iou, 4)
        }
    else:
        fps_results[name] = {"latency": None, "FPS": None, "IoU": None}
        print(f"⚠️ {name}: No valid frames processed.")

    print(f"✅ {name}: Processed {valid_frames} valid frames.")

# ----------------------------
# Report
# ----------------------------

print("\n📊 FPS, Latency, and Accuracy (IoU) Results:")
for model, metrics in fps_results.items():
    print(f"{model}: Latency = {metrics['latency']}s | FPS = {metrics['FPS']} | IoU = {metrics['IoU']}")



🚀 Running evaluation for YOLOv8n-seg

0: 480x640 (no detections), 118.5ms
Speed: 11.7ms preprocess, 118.5ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 Burss, 67.9ms
Speed: 4.3ms preprocess, 67.9ms inference, 4.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 Burss, 22.9ms
Speed: 9.4ms preprocess, 22.9ms inference, 8.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Weld, 26.4ms
Speed: 3.5ms preprocess, 26.4ms inference, 8.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Weld, 28.1ms
Speed: 2.9ms preprocess, 28.1ms inference, 7.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Weld, 28.6ms
Speed: 3.1ms preprocess, 28.6ms inference, 7.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Weld, 24.1ms
Speed: 3.6ms preprocess, 24.1ms inference, 3.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Weld, 27.2ms
Speed: 3.2ms preprocess, 27.2ms inference, 9.5ms post

In [12]:
import os
import time
import cv2
import torch
import numpy as np
from glob import glob

# ----------------------------
# Polygon to Mask Conversion
# ----------------------------

def load_yolo_segmentation_mask(label_path, img_shape):
    h, w = img_shape[:2]
    mask = np.zeros((h, w), dtype=np.uint8)

    if not os.path.exists(label_path):
        return mask

    with open(label_path, "r") as f:
        lines = f.readlines()

    for line in lines:
        parts = line.strip().split()
        if len(parts) < 7:  # class + at least 3 points (x1 y1 x2 y2 x3 y3)
            continue
        coords = np.array(list(map(float, parts[1:])))
        coords = coords.reshape(-1, 2)
        coords[:, 0] *= w  # x
        coords[:, 1] *= h  # y
        polygon = coords.astype(np.int32)
        cv2.fillPoly(mask, [polygon], color=1)

    return mask

# ----------------------------
# Inference Wrappers
# ----------------------------

def measure_model(model_name, inference_fn, frame):
    start = time.time()
    output = inference_fn(frame)
    latency = time.time() - start
    return latency, output

def infer_maskrcnn(frame):
    with torch.no_grad():
        img_tensor = transform(frame).to(device)
        outputs = mask_rcnn([img_tensor])[0]
        if 'masks' in outputs and len(outputs['masks']) > 0:
            mask = outputs['masks'][0, 0] > 0.5
        else:
            mask = torch.zeros((480, 640), dtype=torch.uint8)
        return mask.cpu().numpy()

def infer_deeplab(frame):
    with torch.no_grad():
        img_tensor = transform(frame).unsqueeze(0).to(device)
        out = deeplab(img_tensor)['out']
        mask = torch.sigmoid(out)[0, 0] > 0.5
        return mask.cpu().numpy()

def infer_unet(frame):
    with torch.no_grad():
        img_tensor = transform(frame).unsqueeze(0).to(device)
        out = unet(img_tensor)
        mask = torch.sigmoid(out)[0, 0] > 0.5
        return mask.cpu().numpy()

def infer_yolov8(frame):
    results = yolo(frame)
    if results[0].masks is not None:
        masks = results[0].masks.data
        combined_mask = torch.any(masks, dim=0) if masks.ndim == 3 else masks[0]
    else:
        combined_mask = torch.zeros((480, 640), dtype=torch.uint8)
    return combined_mask.cpu().numpy()

# ----------------------------
# Metric Computation
# ----------------------------

def compute_iou(pred_mask, true_mask):
    pred = pred_mask.astype(bool)
    true = true_mask.astype(bool)
    intersection = np.logical_and(pred, true).sum()
    union = np.logical_or(pred, true).sum()
    return intersection / union if union != 0 else 1.0

def compute_classification_metrics(pred_mask, true_mask):
    pred = pred_mask.astype(bool).flatten()
    true = true_mask.astype(bool).flatten()

    TP = np.logical_and(pred, true).sum()
    FP = np.logical_and(pred, ~true).sum()
    FN = np.logical_and(~pred, true).sum()

    precision = TP / (TP + FP) if (TP + FP) > 0 else 1.0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 1.0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

    return precision, recall, f1_score

# ----------------------------
# Dataset Paths
# ----------------------------

val_image_dir = r"C:\Users\navlog\Dataset_7\val\images"
val_label_dir = r"C:\Users\navlog\Dataset_7\val\labels"

image_paths = sorted(glob(os.path.join(val_image_dir, "*.jpg")))  # or .png

# ----------------------------
# Model Definitions
# ----------------------------

fps_results = {}

model_runs = {
    'YOLOv8n-seg': infer_yolov8,
    'Mask R-CNN': infer_maskrcnn,
    'DeepLabV3+': infer_deeplab,
    'U-Net': infer_unet
}

# ----------------------------
# Evaluation Loop
# ----------------------------

for name, fn in model_runs.items():
    print(f"\n🚀 Running evaluation for {name}")
    total_latency = total_iou = 0
    total_precision = total_recall = total_f1 = 0
    valid_frames = 0

    for i, img_path in enumerate(image_paths[:50]):
        frame = cv2.imread(img_path)
        frame = cv2.resize(frame, (640, 480))

        base_name = os.path.splitext(os.path.basename(img_path))[0]
        label_path = os.path.join(val_label_dir, base_name + ".txt")

        gt_mask = load_yolo_segmentation_mask(label_path, frame.shape)
        latency, pred_mask = measure_model(name, fn, frame)

        iou = compute_iou(pred_mask, gt_mask)
        precision, recall, f1_score = compute_classification_metrics(pred_mask, gt_mask)

        total_latency += latency
        total_iou += iou
        total_precision += precision
        total_recall += recall
        total_f1 += f1_score
        valid_frames += 1

    if valid_frames > 0 and total_latency > 0:
        avg_latency = total_latency / valid_frames
        fps_results[name] = {
            "Latency (s)": round(avg_latency, 4),
            "FPS": round(1 / avg_latency, 2),
            "IoU": round(total_iou / valid_frames, 4),
            "Precision": round(total_precision / valid_frames, 4),
            "Recall": round(total_recall / valid_frames, 4),
            "F1-score": round(total_f1 / valid_frames, 4)
        }
    else:
        fps_results[name] = {
            "Latency (s)": None,
            "FPS": None,
            "IoU": None,
            "Precision": None,
            "Recall": None,
            "F1-score": None
        }

    print(f"✅ {name}: Processed {valid_frames} valid frames.")

# ----------------------------
# Final Report
# ----------------------------

print("\n📊 Evaluation Summary (Latency, FPS, IoU, Precision, Recall, F1-score):")
for model, metrics in fps_results.items():
    print(f"{model}: "
          f"Latency = {metrics['Latency (s)']}s | "
          f"FPS = {metrics['FPS']} | "
          f"IoU = {metrics['IoU']} | "
          f"Precision = {metrics['Precision']} | "
          f"Recall = {metrics['Recall']} | "
          f"F1-score = {metrics['F1-score']}")



🚀 Running evaluation for YOLOv8n-seg

0: 480x640 (no detections), 84.0ms
Speed: 40.3ms preprocess, 84.0ms inference, 15.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 Burss, 72.0ms
Speed: 5.4ms preprocess, 72.0ms inference, 11.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 Burss, 66.5ms
Speed: 4.3ms preprocess, 66.5ms inference, 5.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Weld, 27.5ms
Speed: 3.4ms preprocess, 27.5ms inference, 10.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Weld, 28.3ms
Speed: 3.6ms preprocess, 28.3ms inference, 8.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Weld, 27.9ms
Speed: 3.5ms preprocess, 27.9ms inference, 6.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Weld, 28.6ms
Speed: 3.6ms preprocess, 28.6ms inference, 8.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Weld, 28.1ms
Speed: 3.1ms preprocess, 28.1ms inference, 9.7ms pos

In [9]:
import matplotlib.pyplot as plt

models = list(fps_results.keys())
latencies = [fps_results[m]["latency"] for m in models]
fps_values = [fps_results[m]["FPS"] for m in models]
ious = [fps_results[m]["IoU"] for m in models]

# Plot 1: Latency
plt.figure(figsize=(8, 5))
plt.bar(models, latencies, color='skyblue')
plt.title("Latency per Model")
plt.ylabel("Latency (s)")
plt.xlabel("Model")
plt.grid(axis='y')
plt.tight_layout()
plt.savefig("latency_per_model.png")
plt.show()

# Plot 2: FPS
plt.figure(figsize=(8, 5))
plt.bar(models, fps_values, color='lightgreen')
plt.title("FPS per Model")
plt.ylabel("Frames Per Second")
plt.xlabel("Model")
plt.grid(axis='y')
plt.tight_layout()
plt.savefig("fps_per_model.png")
plt.show()

# Plot 3: IoU
plt.figure(figsize=(8, 5))
plt.bar(models, ious, color='salmon')
plt.title("IoU per Model")
plt.ylabel("Intersection over Union")
plt.xlabel("Model")
plt.grid(axis='y')
plt.tight_layout()
plt.savefig("iou_per_model.png")
plt.show()


<Figure size 800x500 with 1 Axes>

<Figure size 800x500 with 1 Axes>

<Figure size 800x500 with 1 Axes>

In [10]:
import matplotlib.pyplot as plt

# Prepare data for plotting
models = list(fps_results.keys())
latencies = [fps_results[m]['latency'] for m in models]
fps_values = [fps_results[m]['FPS'] for m in models]
ious = [fps_results[m]['IoU'] for m in models]

x = np.arange(len(models))
width = 0.25

plt.figure(figsize=(10, 6))
plt.bar(x - width, latencies, width, label='Latency (s)')
plt.bar(x, fps_values, width, label='FPS')
plt.bar(x + width, ious, width, label='IoU')

plt.xlabel("Model")
plt.ylabel("Value")
plt.title("Model Comparison: Latency, FPS, IoU")
plt.xticks(x, models)
plt.legend()
plt.grid(True, axis='y')
plt.tight_layout()
plt.savefig("model_comparison_metrics.png")  # Optional: Save the plot
plt.show()


<Figure size 1000x600 with 1 Axes>

In [11]:
import csv

csv_path = "model_segmentation_metrics.csv"
with open(csv_path, mode='w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["Model", "Latency (s)", "FPS", "IoU"])
    for model, metrics in fps_results.items():
        writer.writerow([
            model,
            metrics['latency'],
            metrics['FPS'],
            metrics['IoU']
        ])

print(f"\n📁 Metrics saved to: {csv_path}")



📁 Metrics saved to: model_segmentation_metrics.csv
